In [ ]:
%serialconnect to --port COM4

In [ ]:
import os 
print(os.listdir())

In [ ]:
%disconnect

In [ ]:
%esptool --port COM12 erase
#esptool --chip esp32 --port COM11 erase_flash

In [ ]:
# esptool --port com12 --baud 460800 write_flash --flash_size=detect -fm dio 0 esp8266-20190125-v1.10.bin
# esptool --port com11 --baud 460800 write_flash --flash_size=detect -fm dio 0 esp32-20190422-v1.10-290-g8402c26cf.bin

In [ ]:
%serialconnect to --port COM11

In [ ]:
import os
print(os.listdir())

In [ ]:
%sendtofile ap.txt
JUMBOAP
0953313123

In [ ]:
%sendtofile main.py
# -*- coding: utf-8 -*-

import machine
import network
import time
import os

#- check ap config file
AP_SSID = 'upy'
AP_PWD = '1234'
ap_config = None
ap_config_fn = 'ap.txt'
if ap_config_fn in os.listdir():
    print('ap config here!')
    f = open(ap_config_fn)
    ap_config = f.read()
    f.close()
if ap_config:
    print( ('ap_config:', ap_config))
    ap_config = ap_config.split('\n')
    AP_SSID = ap_config[0].strip()
    AP_PWD = ap_config[1].strip()
print('line to: ', (AP_SSID, AP_PWD))

#-- 連到AP 為Station
sta_if = network.WLAN(network.STA_IF)
sta_if.active(True)
sta_if.connect(AP_SSID, AP_PWD)
sta_if.isconnected()
for i in range(20):
    time.sleep(0.5)
    if sta_if.isconnected():
        break
sta_ip = ''
if sta_if.isconnected():
    print('connected!  --> ', sta_if.ifconfig())
    sta_ip = sta_if.ifconfig()[0]
else:
    print('not connected!  --> ', sta_if.ifconfig())

#-- 當AP，並指定
uid = machine.unique_id()
#ap_if.ifconfig()
# ('192.168.4.1', '255.255.255.0', '192.168.4.1', '192.168.43.1')
# (ip, mask, gateway, dns)
my_sn = '%02X-%02X-%02X-%02X' %(uid[0], uid[1], uid[2], uid[3])

#- Change name/password/ip of ESP8266's AP:
ap_if = network.WLAN(network.AP_IF)
#ap_if.ifconfig([my_ip, my_mask, my_gw, my_dns])

my_ssid = 'upy_%s_%s' %(my_sn, sta_ip)
#ap_if.config(essid = my_ssid)#改ssid，馬上生效
ap_if.config(essid=my_ssid, authmode=network.AUTH_WPA_WPA2_PSK, password="12345678")
#DHCP 功能micropython預設就有，不用設定
#AP的IP，每次重開都會回到預設值，因此需要開機時就設定
#一般是配給AP ip的下一號ip


import socket
from machine import Pin
from machine import PWM
import dht
#from hcsr04 import HCSR04

def Tune(buzzer, freq=262, t=500, duty=512):
    ''' t: ms '''
    if freq==0:
        buzzer.duty(0)
    else:
        buzzer.duty(duty)
    buzzer.freq(freq)
    time.sleep_ms(t)


# PIN Define:
D0 = 16
D1 = 5  #PWM
D2 = 4  #PWM
D3 = 0  #PWM
D4 = 2  #PWM, #Led on board
D5 = 14 #PWM
D6 = 12 #PWM
D7 = 13 #PWM
D8 = 15 #PWM

#Setup PINS
led = machine.Pin(0, machine.Pin.OUT)

# buzzer
NOTE_C4=262
NOTE_D4=294
NOTE_E4=330
NOTE_F4=349
NOTE_G4=392
NOTE_A4=440
NOTE_B4=494
buzzer = PWM(Pin(D2), freq=1000, duty=0)


# th_sensor
th_sensor = dht.DHT11(Pin(2))


#Setup Socket WebServer
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind(('0.0.0.0', 80))
s.listen(5)
while True:
    conn, addr = s.accept()
    print("Got a connection from %s" % str(addr))
    request = conn.recv(1024)
    print("Content = %s" % str(request))

    request = str(request)
    led_on = request.find('GET /?LED=ON')
    led_off = request.find('GET /?LED=OFF')
    buzzer_on = request.find('GET /?buzzer=on')
    buzzer_off = request.find('GET /?buzzer=off')
    th_sensor_read = request.find('GET /?th_sensor=read')


    if led_on >= 0:
        print('TURN Led ON')
        led.value(0)
    if led_off >= 0:
        print('TURN Led OFF')
        led.value(1)
    if buzzer_on >= 0:
        print('buzzer on')
        Tune(buzzer, NOTE_C4, 10)
    if buzzer_off >= 0:
        print('buzzer off')
        Tune(buzzer, 0, 10)

    my_t = None
    if th_sensor_read >= 0:
        print('th_sensor read')
        th_sensor.measure()
        my_t = th_sensor.temperature()
        print('T=', th_sensor.temperature())
        print('H=', th_sensor.humidity())

    f = open('webtool.html')

    while(1):
        html = f.read(1024)

        if my_t:
            html = html.replace('T=?degree', 'T=%d degree' %(my_t))
        else:
            html = html.replace('T=?degree', '')
        #conn.send(html)
        conn.sendall(html) #改用send all就不會有資料傳一半的問題
        if(len(html)<=0):
            break
    f.close()
    conn.close()

In [ ]:
%sendtofile webtool.html
<!DOCTYPE html>
<html>
<head> <title>upy webserver</title> </head>
<center><h2>upy webserver</h2></center>
<h3>
webREPL: <a href="http://micropython.org/webrepl/" target="_blank">webrepl</a>
<br>
Doc: <a href="https://docs.micropython.org/en/latest/esp8266/esp8266/quickref.html" target="_blank">Quick reference</a>
</h3>
<hr>

<form>
<b>LED (D4):</b><br>
<button name="LED" value="ON" type="submit">ON</button><br>
<button name="LED" value="OFF" type="submit">OFF</button><br>
</form>

<form>
<b>T/H sensor (D1):</b><br>
<button name="th_sensor" value="read" type="submit">Read</button><br>
T=?degree
H=?unit
</form>

<hr>
GPIO Info. :<br>
<pre>
D0 = 16
D1 = 5  #PWM
D2 = 4  #PWM
D3 = 0  #PWM
D4 = 2  #PWM, #Led on board
D5 = 14 #PWM
D6 = 12 #PWM
D7 = 13 #PWM
D8 = 15 #PWM
</pre>

</html>

In [ ]:
%sendtofile webrepl_cfg.py
PASS = '1234'

In [ ]:
%rebootdevice

In [ ]:
%sendtofile boot.py
# This file is executed on every boot (including wake-boot from deepsleep)
#import esp
#esp.osdebug(None)
import uos, machine
#uos.dupterm(None, 1) # disable REPL on UART(0)
import gc
#import webrepl
#webrepl.start()
gc.collect()

In [ ]:
%sendtofile boot.py
import uos, machine
import gc
gc.collect()

In [ ]:
%serialconnect to --port COM4

In [ ]:
import time
from machine import Pin

led = machine.Pin(15, machine.Pin.OUT)
for _ in range(10):
    led.value(1)
    time.sleep(0.5)
    led.value(0)
    

In [ ]:
led.value(0)

In [ ]:
%sendtofile blink.py
from machine import Pin
import time

p2 = Pin(15, Pin.OUT)

for i in range(3):
    p2.value(0)
    time.sleep(1)
    p2.value(1)
    time.sleep(1)

In [ ]:
%sendtofile read_dht.py
from machine import Pin
import dht
sensor = dht.DHT11(Pin(5))
sensor.measure()
print(sensor.temperature())
print(sensor.humidity())

In [ ]:
import read_dht

In [ ]:
f = open('blink.py')
print(f.read())

In [ ]:
import blink

### [官網](http://docs.micropython.org/en/latest/esp8266/quickref.html#adc-analog-to-digital-conversion)

In [ ]:
from machine import ADC

adc = ADC(0)                   # create ADC object on ADC pin
print(adc.read())              # read value, 0-1024

In [ ]:
%sendtofile read_adc.py
from machine import ADC
adc = ADC(0)            # create ADC object on ADC pin
print(adc.read())              # read value, 0-1024

In [ ]:
import read_adc

In [3]:
import webrepl
webrepl.start()

WebREPL daemon started on ws://192.168.4.1:8266
WebREPL daemon started on ws://192.168.43.121:8266
Started webrepl in normal mode


In [2]:
%serialconnect to --port COM4

Connecting to --port=COM4 --baud=115200 
Ready.


In [ ]:
%disconnect

In [1]:
%websocketconnect --password 1234 ws://192.168.43.121:8266

WebSocket OSError [WinError 10061] 無法連線，因為目標電腦拒絕連線。


In [ ]:
import read_adc

In [ ]:
from machine import Pin
from machine import ADC
import time
p2 = Pin(15, Pin.OUT)
adc = ADC(0)            # create ADC object on ADC pin
while True:
     if adc.read() > 500:
            p2.value(1)
     time.sleep(2)
     p2.value(0)

..............................................

In [ ]:
%disconnect

In [ ]:
# ampy --port COM4 ls
# ampy --port COM4 get boot.py > boot.py
# ampy --port COM4 put readDHT.py
# ampy --port COM4 run readDHT.py